In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
train.shape

(7613, 5)

In [4]:
test = pd.read_csv('test.csv')
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
test.shape

(3263, 4)

In [6]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [7]:
test.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [8]:
from bs4 import BeautifulSoup

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [10]:
ps = PorterStemmer()

In [11]:
from nltk.stem import WordNetLemmatizer

lem = WordNetLemmatizer()

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer()

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train['text'], train['target'], test_size = 0.3, random_state = 0)

In [14]:
corpus_train = []
corpus_test  = []

for i in range(x_train.shape[0]):
    soup = BeautifulSoup(x_train.iloc[i], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_train.append(review)

In [15]:
for j in range(x_test.shape[0]):
    soup = BeautifulSoup(x_test.iloc[j], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_test.append(review)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3))

tfidf_vec_train = tfidf_vec.fit_transform(corpus_train)
tfidf_vec_test = tfidf_vec.transform(corpus_test)

In [17]:
from sklearn.svm import LinearSVC
linear_svc = LinearSVC(C=0.5, random_state=42)

In [18]:
linear_svc.fit(tfidf_vec_train, y_train)

LinearSVC(C=0.5, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [19]:
predict = linear_svc.predict(tfidf_vec_test)

In [20]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [21]:
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       0.83      0.82      0.82      1338
           1       0.75      0.77      0.76       946

    accuracy                           0.80      2284
   macro avg       0.79      0.79      0.79      2284
weighted avg       0.80      0.80      0.80      2284



In [22]:
print(confusion_matrix(y_test, predict))

[[1092  246]
 [ 220  726]]


In [23]:
print("Accuracy:", accuracy_score(y_test, predict))

Accuracy: 0.7959719789842382


In [24]:
x_train = train['text']
y_train = train['target']
x_test = test['text']

In [25]:
corpus_train = []
corpus_test  = []

for i in range(x_train.shape[0]):
    soup = BeautifulSoup(x_train.iloc[i], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_train.append(review)

In [26]:
for j in range(x_test.shape[0]):
    soup = BeautifulSoup(x_test.iloc[j], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_test.append(review)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3))

tfidf_vec_train = tfidf_vec.fit_transform(corpus_train)
tfidf_vec_test = tfidf_vec.transform(corpus_test)

In [35]:
from catboost import CatBoostClassifier
lor = CatBoostClassifier()
lor.fit(tfidf_vec_train, y_train)
y_pred = lor.predict(tfidf_vec_test)
solution = pd.DataFrame({'id': test.id, 'target': y_pred})
solution.to_csv('Solution2.csv', index = False)

Learning rate set to 0.024511
0:	learn: 0.6902101	total: 565ms	remaining: 9m 24s
1:	learn: 0.6868095	total: 956ms	remaining: 7m 57s
2:	learn: 0.6844171	total: 1.33s	remaining: 7m 21s
3:	learn: 0.6815060	total: 1.69s	remaining: 7m
4:	learn: 0.6792228	total: 2.06s	remaining: 6m 49s
5:	learn: 0.6762471	total: 2.43s	remaining: 6m 42s
6:	learn: 0.6740390	total: 2.81s	remaining: 6m 38s
7:	learn: 0.6714917	total: 3.17s	remaining: 6m 33s
8:	learn: 0.6692585	total: 3.54s	remaining: 6m 30s
9:	learn: 0.6670478	total: 3.91s	remaining: 6m 27s
10:	learn: 0.6650220	total: 4.28s	remaining: 6m 25s
11:	learn: 0.6628925	total: 4.64s	remaining: 6m 22s
12:	learn: 0.6606613	total: 5.01s	remaining: 6m 20s
13:	learn: 0.6585163	total: 5.37s	remaining: 6m 18s
14:	learn: 0.6568791	total: 5.74s	remaining: 6m 16s
15:	learn: 0.6549635	total: 6.1s	remaining: 6m 15s
16:	learn: 0.6536055	total: 6.46s	remaining: 6m 13s
17:	learn: 0.6522332	total: 6.83s	remaining: 6m 12s
18:	learn: 0.6503276	total: 7.19s	remaining: 6m 1

158:	learn: 0.5606785	total: 1m 2s	remaining: 5m 29s
159:	learn: 0.5603579	total: 1m 2s	remaining: 5m 29s
160:	learn: 0.5601548	total: 1m 3s	remaining: 5m 29s
161:	learn: 0.5598062	total: 1m 3s	remaining: 5m 28s
162:	learn: 0.5596270	total: 1m 3s	remaining: 5m 28s
163:	learn: 0.5592974	total: 1m 4s	remaining: 5m 28s
164:	learn: 0.5589654	total: 1m 4s	remaining: 5m 28s
165:	learn: 0.5587511	total: 1m 5s	remaining: 5m 27s
166:	learn: 0.5584623	total: 1m 5s	remaining: 5m 27s
167:	learn: 0.5581514	total: 1m 6s	remaining: 5m 26s
168:	learn: 0.5578067	total: 1m 6s	remaining: 5m 26s
169:	learn: 0.5574411	total: 1m 6s	remaining: 5m 26s
170:	learn: 0.5572861	total: 1m 7s	remaining: 5m 25s
171:	learn: 0.5569714	total: 1m 7s	remaining: 5m 25s
172:	learn: 0.5567621	total: 1m 7s	remaining: 5m 25s
173:	learn: 0.5564694	total: 1m 8s	remaining: 5m 24s
174:	learn: 0.5561313	total: 1m 8s	remaining: 5m 23s
175:	learn: 0.5557258	total: 1m 9s	remaining: 5m 23s
176:	learn: 0.5554608	total: 1m 9s	remaining: 

311:	learn: 0.5222934	total: 2m	remaining: 4m 25s
312:	learn: 0.5221041	total: 2m	remaining: 4m 24s
313:	learn: 0.5218495	total: 2m	remaining: 4m 24s
314:	learn: 0.5216706	total: 2m 1s	remaining: 4m 23s
315:	learn: 0.5214662	total: 2m 1s	remaining: 4m 23s
316:	learn: 0.5212436	total: 2m 1s	remaining: 4m 22s
317:	learn: 0.5210178	total: 2m 2s	remaining: 4m 22s
318:	learn: 0.5208107	total: 2m 2s	remaining: 4m 21s
319:	learn: 0.5204560	total: 2m 2s	remaining: 4m 21s
320:	learn: 0.5202875	total: 2m 3s	remaining: 4m 20s
321:	learn: 0.5201125	total: 2m 3s	remaining: 4m 20s
322:	learn: 0.5198824	total: 2m 3s	remaining: 4m 19s
323:	learn: 0.5197076	total: 2m 4s	remaining: 4m 19s
324:	learn: 0.5194763	total: 2m 4s	remaining: 4m 18s
325:	learn: 0.5192566	total: 2m 5s	remaining: 4m 18s
326:	learn: 0.5190169	total: 2m 5s	remaining: 4m 18s
327:	learn: 0.5187653	total: 2m 5s	remaining: 4m 17s
328:	learn: 0.5185390	total: 2m 6s	remaining: 4m 17s
329:	learn: 0.5183463	total: 2m 6s	remaining: 4m 16s
33

464:	learn: 0.4876067	total: 2m 59s	remaining: 3m 26s
465:	learn: 0.4873494	total: 3m	remaining: 3m 26s
466:	learn: 0.4871623	total: 3m	remaining: 3m 25s
467:	learn: 0.4869223	total: 3m	remaining: 3m 25s
468:	learn: 0.4867429	total: 3m 1s	remaining: 3m 25s
469:	learn: 0.4865609	total: 3m 1s	remaining: 3m 24s
470:	learn: 0.4863854	total: 3m 2s	remaining: 3m 24s
471:	learn: 0.4861307	total: 3m 2s	remaining: 3m 24s
472:	learn: 0.4858633	total: 3m 2s	remaining: 3m 23s
473:	learn: 0.4856899	total: 3m 3s	remaining: 3m 23s
474:	learn: 0.4855190	total: 3m 3s	remaining: 3m 23s
475:	learn: 0.4852332	total: 3m 4s	remaining: 3m 22s
476:	learn: 0.4849767	total: 3m 4s	remaining: 3m 22s
477:	learn: 0.4847542	total: 3m 4s	remaining: 3m 21s
478:	learn: 0.4845745	total: 3m 5s	remaining: 3m 21s
479:	learn: 0.4843743	total: 3m 5s	remaining: 3m 21s
480:	learn: 0.4840976	total: 3m 6s	remaining: 3m 20s
481:	learn: 0.4839224	total: 3m 6s	remaining: 3m 20s
482:	learn: 0.4836794	total: 3m 7s	remaining: 3m 20s
4

617:	learn: 0.4578911	total: 3m 53s	remaining: 2m 24s
618:	learn: 0.4577171	total: 3m 54s	remaining: 2m 24s
619:	learn: 0.4575259	total: 3m 54s	remaining: 2m 23s
620:	learn: 0.4573138	total: 3m 54s	remaining: 2m 23s
621:	learn: 0.4571928	total: 3m 55s	remaining: 2m 22s
622:	learn: 0.4570642	total: 3m 55s	remaining: 2m 22s
623:	learn: 0.4568837	total: 3m 55s	remaining: 2m 22s
624:	learn: 0.4567073	total: 3m 56s	remaining: 2m 21s
625:	learn: 0.4566048	total: 3m 56s	remaining: 2m 21s
626:	learn: 0.4564173	total: 3m 56s	remaining: 2m 20s
627:	learn: 0.4563330	total: 3m 57s	remaining: 2m 20s
628:	learn: 0.4562162	total: 3m 57s	remaining: 2m 20s
629:	learn: 0.4560321	total: 3m 57s	remaining: 2m 19s
630:	learn: 0.4558517	total: 3m 58s	remaining: 2m 19s
631:	learn: 0.4556987	total: 3m 58s	remaining: 2m 18s
632:	learn: 0.4555413	total: 3m 58s	remaining: 2m 18s
633:	learn: 0.4553345	total: 3m 59s	remaining: 2m 18s
634:	learn: 0.4550829	total: 3m 59s	remaining: 2m 17s
635:	learn: 0.4549067	total:

771:	learn: 0.4350588	total: 4m 45s	remaining: 1m 24s
772:	learn: 0.4349085	total: 4m 45s	remaining: 1m 23s
773:	learn: 0.4348569	total: 4m 46s	remaining: 1m 23s
774:	learn: 0.4346615	total: 4m 46s	remaining: 1m 23s
775:	learn: 0.4344338	total: 4m 46s	remaining: 1m 22s
776:	learn: 0.4343108	total: 4m 47s	remaining: 1m 22s
777:	learn: 0.4341520	total: 4m 47s	remaining: 1m 22s
778:	learn: 0.4340312	total: 4m 47s	remaining: 1m 21s
779:	learn: 0.4338927	total: 4m 48s	remaining: 1m 21s
780:	learn: 0.4337869	total: 4m 48s	remaining: 1m 20s
781:	learn: 0.4334924	total: 4m 48s	remaining: 1m 20s
782:	learn: 0.4333307	total: 4m 49s	remaining: 1m 20s
783:	learn: 0.4332291	total: 4m 49s	remaining: 1m 19s
784:	learn: 0.4330825	total: 4m 50s	remaining: 1m 19s
785:	learn: 0.4329511	total: 4m 50s	remaining: 1m 19s
786:	learn: 0.4327802	total: 4m 50s	remaining: 1m 18s
787:	learn: 0.4326867	total: 4m 51s	remaining: 1m 18s
788:	learn: 0.4325307	total: 4m 51s	remaining: 1m 17s
789:	learn: 0.4323394	total:

927:	learn: 0.4159319	total: 5m 37s	remaining: 26.2s
928:	learn: 0.4157557	total: 5m 38s	remaining: 25.9s
929:	learn: 0.4155733	total: 5m 38s	remaining: 25.5s
930:	learn: 0.4154536	total: 5m 38s	remaining: 25.1s
931:	learn: 0.4153062	total: 5m 39s	remaining: 24.8s
932:	learn: 0.4152424	total: 5m 39s	remaining: 24.4s
933:	learn: 0.4151932	total: 5m 39s	remaining: 24s
934:	learn: 0.4150857	total: 5m 40s	remaining: 23.7s
935:	learn: 0.4149257	total: 5m 40s	remaining: 23.3s
936:	learn: 0.4148047	total: 5m 41s	remaining: 22.9s
937:	learn: 0.4147368	total: 5m 41s	remaining: 22.6s
938:	learn: 0.4146167	total: 5m 41s	remaining: 22.2s
939:	learn: 0.4144853	total: 5m 42s	remaining: 21.8s
940:	learn: 0.4144556	total: 5m 42s	remaining: 21.5s
941:	learn: 0.4142902	total: 5m 42s	remaining: 21.1s
942:	learn: 0.4142223	total: 5m 43s	remaining: 20.7s
943:	learn: 0.4141038	total: 5m 43s	remaining: 20.4s
944:	learn: 0.4139558	total: 5m 43s	remaining: 20s
945:	learn: 0.4138080	total: 5m 44s	remaining: 19.